# t-SNE (Amazon fine food dataset)

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
import sqlite3
import nltk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer


import warnings
warnings.filterwarnings('ignore')

In [3]:
con = sqlite3.connect('/home/akash/Downloads/amazon/database.sqlite')

In [4]:
filtered_data = pd.read_sql_query('''select * from Reviews where Score != 3''', con)
filtered_data.shape

(525814, 10)

In [5]:
def partition(x):
    if x < 3:
        return 'negative'
    else:
        return 'positive'

In [6]:
actual_score = filtered_data['Score']
posneg = actual_score.map(partition)
filtered_data['Score'] = posneg
print(filtered_data.shape)
filtered_data.head()

(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [7]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [8]:
sorted_data = filtered_data.sort_values('ProductId', axis=0, ascending= True, inplace= False, kind= 'quicksort', na_position='last')
final = sorted_data.drop_duplicates(subset={'UserId', 'ProfileName', 'Time', 'Text'}, keep='first', inplace=False)
final.shape

(364173, 10)

In [9]:
final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]

In [10]:
final.shape

(364171, 10)

# pre processing

In [11]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

i=0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [12]:
stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

{'no', 'about', "wouldn't", 'before', 'under', 'me', 'weren', 'theirs', 'on', 'if', 'below', 'not', 'wasn', 'being', 'yourselves', "shan't", "haven't", 'mustn', 'we', 'can', "that'll", "aren't", 'off', 'had', 'few', "hasn't", 'i', 'don', 'of', 'out', "hadn't", 'whom', 'yourself', 'won', 'more', "doesn't", 'herself', 'a', 'were', 'which', 'or', 'o', 'against', 'and', 'after', 'm', "isn't", 'does', 'been', 'what', "should've", 'now', "couldn't", 'couldn', 'ma', 'll', 'some', 's', "you'll", 'aren', 'am', 'are', 'wouldn', 'than', 'himself', 'his', 'they', 'having', 'from', 'ourselves', 'mightn', 'for', 't', 'between', 'when', 'd', "won't", 'where', 'once', 'to', 'most', 'will', "wasn't", 'do', 'have', 'until', 'again', "she's", 'was', "mustn't", "don't", "you'd", "you're", 'those', 'doesn', 'into', 'by', 'as', "it's", 'while', 'y', 'just', 'did', 'hadn', 'at', 'my', 'these', 've', "weren't", 'has', 'above', 'why', 'then', 'how', 'very', 'with', 'she', "didn't", 'its', "shouldn't", 'hers', 

In [13]:
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [14]:
final['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 
final['CleanedText']=final['CleanedText'].str.decode("utf-8")

In [15]:
final.to_csv('final.csv')

In [22]:
print(final.head(3)) #below the processed review can be seen in the CleanedText Column 


# store final table into an SQlLite table for future.
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn,  schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

            Id   ProductId          UserId            ProfileName  \
138706  150524  0006641040   ACITT7DI6IDDL        shari zychinski   
138688  150506  0006641040  A2IW4PEEKO2R0U                  Tracy   
138689  150507  0006641040  A1S4A3IQ2MU7V4  sally sue "sally sue"   

        HelpfulnessNumerator  HelpfulnessDenominator     Score        Time  \
138706                     0                       0  positive   939340800   
138688                     1                       1  positive  1194739200   
138689                     1                       1  positive  1191456000   

                                           Summary  \
138706                   EVERY book is educational   
138688  Love the book, miss the hard cover version   
138689               chicken soup with rice months   

                                                     Text  \
138706  this witty little book makes my son laugh at l...   
138688  I grew up reading these Sendak books, and watc...   
138689  Th

# BoW

In [23]:
count_vect = CountVectorizer()
final_counts = count_vect.fit_transform(final["CleanedText"].values)
print('Type: ', type(final_counts))
print('shape: ', final_counts.get_shape)
print('No. of unique words: ', final_counts.get_shape()[1])

Type:  <class 'scipy.sparse.csr.csr_matrix'>
shape:  <bound method spmatrix.get_shape of <364171x115281 sparse matrix of type '<class 'numpy.int64'>'
	with 19341760 stored elements in Compressed Sparse Row format>>
No. of unique words:  115281


In [ ]:
from sklearn.manifold import TSNE
data_2k = final_counts[0:2000,:]
top_2k= data_2k.toarray()
labels=final['Score']
labels_2k = labels[0:2000]

model = TSNE(n_components=2, random_state=0, perplexity=50, n_iter=1000)
tsne_data= model.fit_transform(top_2k)

tsne_data= np.vstack((tsne_data.T, labels_2k)).T
tsne_df= pd.DataFrame(tsne_data, columns=('Dim1','Dim2','label'))

print(tsne_df.head())
print(tsne_df.shape)

sns.FacetGrid(tsne_df, hue='label', size=6).map(plt.scatter, 'Dim1','Dim2').add_legend()
plt.title('1000 iteration with perplexity 50')
plt.show()

In [ ]:
tfidf_vect = TfidfVectorizer()
final_tfidf = tfidf_vect.fit_transform(final['Text'].values)

data_2k = final_tfidf[0:2000,:]
top_2k = data_2k.toarray()
labels = final['Score']
labels_2k = labels[0:2000]

model = TSNE(n_components= 2, random= 0, perplexity= 50)
tsne_data= model.fit_transform(top_2k)

tsne_data = np.vstack((tsne_data.T, label_2k)).T
tsne_df = pd.DataFrame(tsne_data, columns= ('Dim_1', 'Dim_2','label'))

sns.FacetGrid(tsne_df, hue= 'label', size=6).map(plt.scatter, 'Dim_1','Dim_2').add_legend()
plt.title('1000 iteration with perpleity 30')
plt.show()

In [ ]:
tfidf_vect = TfidfVectorizer()
final_tfidf = tfidf_vect.fit_transform(final['Text'].values)

data_4k = final_tfidf[0:4000,:]
top_4k = data_4k.toarray()
labels = final['Score']
labels_4k = labels[0:4000]

model = TSNE(n_components= 2, random= 0, perplexity= 50)
tsne_data= model.fit_transform(top_4k)

tsne_data = np.vstack((tsne_data.T, label_4k)).T
tsne_df = pd.DataFrame(tsne_data, columns= ('Dim_1', 'Dim_2','label'))

sns.FacetGrid(tsne_df, hue= 'label', size=6).map(plt.scatter, 'Dim_1','Dim_2').add_legend()
plt.title('1000 iteration with perpleity 30')
plt.show()